In [150]:
# Training of pRNN ansatz for generation of Symmetric Polynomials

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) #stop displaying tensorflow warnings

import importnb

import numpy as np
import matplotlib.pyplot as plt

with importnb.Notebook('pRNN_homo_sample.ipynb'):
    from pRNN_homo_sample import RNNWavefunction

In [151]:
# Computes local energies given a set of computational basis vectors sampled according to the pRNN wavefunction probability

def symPoly_local_energies(a,b,Nc, samples, model):
    """ Local energies of a system of Nc**2 interacting bosonic oscillators.
    Returns: The local energies that correspond to the "samples"
    Inputs:
    - a,b: define the Gauge generator
    - Nc: rank of the matrix group
    - samples: (numsamples, Nc**2)
    - model: The RNN wavefunction model instance
    """

    numsamples = samples.shape[0]  # Extracts number of samples

    local_energies = np.zeros((numsamples), dtype=np.float64)  # Initialize local energy to zero for each sample
    
    energy_samples= np.zeros((2*Nc,numsamples), dtype=np.float64) # Array of energies to zero for each sample
    queue_samples= np.zeros((1+2*Nc,numsamples,Nc*Nc), dtype=np.float64)  # Array of vector states to zero for each sample
    log_probs= np.zeros((1+2*Nc)*numsamples, dtype=np.float64) # Array of log probs for each vector state for each sample
    
    queue_samples[0]=samples

    hmagnetic = 300 # Magnetic field to tesy symmetry-breaking
    
    # Evaluation of local energy

    for c in range(Nc):  # +1 terms
        samplesT = np.copy(samples) #copy samples to generate new vectors

        energy_samples[c,:]= samplesT[:, (a-1)*Nc+c]
        #print(samplesT)
        #print(energy_samples[c,:])
        
        samplesT[:, (a-1)*Nc+c] -= 1 # One excitation hops away from site (a,c)
        samplesT[:, (b-1)*Nc+c] += 1 # One excitation hops in to site (b,c)
        
        queue_samples[1+c] = samplesT

    for c in range(Nc):  # -1 terms
        samplesT = np.copy(samples)
        
        energy_samples[c,:] = - samplesT[:, c*Nc+(b-1)]
        
        samplesT[:, c*Nc+(b-1)] -= 1 # One excitation hops away from site (c,b)
        samplesT[:, c*Nc+(a-1)] += 1 # One excitation hops in to site (c,a)
        
        queue_samples[1+Nc+c] = samplesT

    # Evaluate log probability of samples and of flipped sample vectors, according to pRNN amplitudes
    
    queue_samples_reshaped = np.reshape(queue_samples, [(1+2*Nc) * numsamples, Nc*Nc])
    len_sigmas = queue_samples_reshaped.shape[0]

    steps = np.ceil(len_sigmas / 25000)  # Maximum of 25000 configurations in batch size for memory reasons

    for i in range(int(steps)):
        cut = slice((i * len_sigmas) // int(steps), ((i + 1) * len_sigmas) // int(steps))
        log_probs[cut] = model.log_probability(queue_samples_reshaped[cut]) # Computes log probabilities slice-by-slice

    log_probs_reshaped = np.reshape(log_probs,[1+2*Nc,numsamples]) # Reshape log_probs putting in line all log probs related to a given sample
    #print("Here: we expect all coinciding: "+str(log_probs_reshaped))
    amplitudes_ratio = np.exp(0.5 * log_probs_reshaped[1:, :] - 0.5 * log_probs_reshaped[0, :]) #Ratio of wavefunction amplitudes for each flipped over unflipped sample

    local_energies += np.sum(energy_samples*amplitudes_ratio,axis=0) # Adds up all local energy terms 

    return local_energies

In [155]:
# Training of pRNN with VMC for energy minimization
# Uses Adam optimizer
# Uses gradient decay

def train_rnn_wavefunction(model, Nc, num_epochs, learning_rate, num_samples):
    """
    Train the RNN wavefunction model to minimize the local energy.
    Parameters:
    - model: RNN-based wavefunction model
    - Nc rank of matrices (Nc*Nc matrix elements =  bosonic oscillators = variables in the polynomial)
    - num_epochs: int, number of training epochs
    - learning_rate: float, learning rate for optimization
    - num_samples: int, total number of samples per epoch
    """
    # Define initial learning rate, decay rate, and decay steps
    initial_learning_rate = learning_rate
    decay_rate = 0.5
    decay_steps = 250  # The step interval at which the learning rate is updated

    # Create an exponential decay schedule
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=False  # If True, decay happens in discrete steps
    )

    # Use the schedule in an optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    energy_vec = [] # Stores avg energies during optimization for plot
    std_vec = [] # Stores std devs during optimization for plot
    avg_magnetization = [] # Stores average spin vector during optimization for plot

    for epoch in range(num_epochs):
        samples = model.sample(num_samples)  # Generate num_samples samples from the RNN wavefunction

        random_array =np.random.rand(Nc*Nc)
        local_energies= np.array([tf.cast(random_array[(a-1)*Nc+b]*symPoly_local_energies(a,b,Nc,samples, model), dtype=tf.float32) for b in range(Nc) for a in range(Nc)]) # Compute local energy for every sample
        
        #local_energies= tf.cast(symPoly_local_energies(2,2,Nc,samples, model), dtype=tf.float32)+0.6*tf.cast(symPoly_local_energies(1,1,Nc,samples, model), dtype=tf.float32) 
 
        with tf.GradientTape() as tape:
            log_probs = model.log_probability(samples)
            loss = tf.abs(tf.abs(tf.reduce_mean(log_probs * local_energies))-tf.abs(tf.reduce_mean(log_probs))*tf.abs(tf.reduce_mean(local_energies))) # Energy minimization loss
            #print("loss function evaluates to: "+str(loss))
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
        avg_energy = np.mean(local_energies)
        std_energy = np.std(local_energies)
        
        energy_vec.append(avg_energy)
        std_vec.append(std_energy)
        avg_magnetization.append(np.mean(tf.reshape(samples,[-1])))
        
        #print(f"Epoch {epoch + 1}/{num_epochs}, Energy: {avg_energy:.6f}, Std Dev: {std_energy:.6f}, Mean Spin: {np.mean(samples):.6f}")

            #if epoch % 50 ==0:
            #    print("Time step = "+str(epoch) )
        #if epoch == num_epochs-1:
            #print(np.mean(samples,axis=0))
        
    return energy_vec, std_vec , avg_magnetization, np.mean(samples,axis=0)

In [159]:
# Define the system size (number of timesteps), number of GRU units, and other parameters
Nc = 2 # Gauge group rank

system_size = Nc*Nc # Number of time steps = spin sites
system_charge =  2 # Polynomial degree (= power of matrix)
units = 20  # Number of (hidden) units in the GRU layer
input_dim = system_charge+1  # Input dimension (e.g., spin-1/2 system)
output_dim = system_charge+1  # Output dimension (e.g., 2 possible states per timestep)
seed = 111


# Generate random samples
numsamples = 50

# Create the RNNWavefunction model
rnn_wavefunction = RNNWavefunction(system_size=Nc*Nc, units=units, input_dim=input_dim, output_dim=output_dim,seed =seed)

#samples = rnn_wavefunction.sample(numsamples)

analysis_vector =np.zeros((Nc*Nc,numsamples))

for i in range(4):
    rnn_wavefunction = RNNWavefunction(system_size=Nc*Nc, units=units, input_dim=input_dim, output_dim=output_dim,seed =seed)
    energies, std_devs, order_param, last_sample = train_rnn_wavefunction(rnn_wavefunction,Nc, num_epochs=100,learning_rate= 0.5,num_samples=numsamples)
    print(last_sample)

[1. 1. 0. 0.]
[0. 0. 0. 2.]
[0.   0.68 0.4  0.92]
[0. 0. 0. 2.]
